<a href="https://colab.research.google.com/github/Amirosein/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile
from zipfile import ZipFile

!pip install tsfresh
from tsfresh import extract_features

!pip install pyunpack
!pip install patool
from pyunpack import Archive

#import warnings
#warnings.filterwarnings('ignore')


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})
SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

> * **Save file function**

In [4]:
def savefile(df,name):
  t = df
  t.to_csv(name, header=True, index=True)
  return 0

# Mount drive

In [5]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **Convert Dat. files to a CSV Function**


In [ ]:
import pandas as pd
from pathlib import Path
import csv
from pathlib import Path

f = open("/content/draft.csv", "x")

# function
def data2csv(path, var, os):
  files = Path(path).rglob("*.dat")

  i = 0

  for file in files:
    datContent = [i.strip().split() for i in open(file).readlines()]
    id = os +'-'+ Path(file).stem
    
    # write it as a new CSV file
    with open("/content/draft.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerows(datContent)

    df = pd.read_csv("/content/draft.csv", names=['time','value','e']) 
    df = df[['time','value']]
    df['id'] = id
    df['kind'] = var

    if i==0:
      data = df
    else:
      data = pd.concat([data,df], axis=0)

    i = i+1
    print(i)

  return data

# Convert and Save data

In [ ]:
Archive('/content/drive/MyDrive/LPV.rar').extractall('/content/drive/MyDrive')


> * **LMC**

In [ ]:
LMC3_I1 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p1', 'I','III')
LMC3_I2 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p2', 'I','III')
LMC3_I3 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p3', 'I','III')
LMC3_I4 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p4', 'I','III')
LMC3_I5 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p5', 'I','III')
LMC3_I6 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p6', 'I','III')
LMC3_I7 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p7', 'I','III')
LMC3_I8 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p8', 'I','III')
LMC3_I9 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p9', 'I','III')
LMC3_I10 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p10', 'I','III')
LMC3_I11 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p11', 'I','III')
LMC3_I12 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p12', 'I','III')
LMC3_I13 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p13', 'I','III')
LMC3_I14 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p14', 'I','III')
LMC3_I15 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p15', 'I','III')
LMC3_I16 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p16', 'I','III')
LMC3_I17 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p17', 'I','III')
LMC3_I18 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/I/p18', 'I','III')

LMC3_I = pd.concat([LMC3_I1,LMC3_I2,LMC3_I3,LMC3_I4,LMC3_I5,LMC3_I6,
                    LMC3_I7,LMC3_I8,LMC3_I9,LMC3_I10,LMC3_I11,LMC3_I12,
                    LMC3_I13,LMC3_I14,LMC3_I15,LMC3_I16,LMC3_I17,LMC3_I18])


In [ ]:
LMC3_V1 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p1', 'V','III')
LMC3_V2 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p2', 'V','III')
LMC3_V3 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p3', 'V','III')
LMC3_V4 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p4', 'V','III')
LMC3_V5 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p5', 'V','III')
LMC3_V6 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p6', 'V','III')
LMC3_V7 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p7', 'V','III')
LMC3_V8 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p8', 'V','III')
LMC3_V9 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p9', 'V','III')
LMC3_V10 = data2csv('/content/drive/MyDrive/LPV/IIILMC/phot/V/p10', 'V','III')

LMC3_V = pd.concat([LMC3_V1,LMC3_V2,LMC3_V3,LMC3_V4,LMC3_V5,LMC3_V6,
                    LMC3_V7,LMC3_V8,LMC3_V9,LMC3_V10])


In [ ]:
LPV_LMC = pd.concat([LMC3_I,LMC3_V])
  
LPV_LMC['class'] = 'LPV'  
savefile(LPV_LMC,'LPV_LMC')
LPV_LMC.head()

,time,value,id,kind,class
0,2171.86025,14.956,III-OGLE-LMC-LPV-00001,I,LPV
1,2183.81956,14.961,III-OGLE-LMC-LPV-00001,I,LPV
2,2184.81907,14.961,III-OGLE-LMC-LPV-00001,I,LPV
3,2188.86966,14.970,III-OGLE-LMC-LPV-00001,I,LPV
4,2191.78070,14.955,III-OGLE-LMC-LPV-00001,I,LPV


> * **SMC**

In [ ]:
SMC3_I1 = data2csv('/content/drive/MyDrive/LPV/III/phot/I/p1', 'I','III')
SMC3_I2 = data2csv('/content/drive/MyDrive/LPV/III/phot/I/p2', 'I','III')
SMC3_I3 = data2csv('/content/drive/MyDrive/LPV/III/phot/I/p3', 'I','III')
SMC3_I4 = data2csv('/content/drive/MyDrive/LPV/III/phot/I/p4', 'I','III')
SMC3_I = pd.concat([SMC3_I1,SMC3_I2,SMC3_I3,SMC3_I4])

savefile(SMC3_I,'SMC3_I')

In [ ]:
SMC3_V1 = data2csv('/content/drive/MyDrive/LPV/III/phot/V/p1', 'V','III')
SMC3_V2 = data2csv('/content/drive/MyDrive/LPV/III/phot/V/p2', 'V','III')
SMC3_V3 = data2csv('/content/drive/MyDrive/LPV/III/phot/V/p3', 'V','III')
SMC3_V4 = data2csv('/content/drive/MyDrive/LPV/III/phot/V/p4', 'V','III')
SMC3_V = pd.concat([SMC3_V1,SMC3_V2,SMC3_V3,SMC3_V4])

savefile(SMC3_V,'SMC3_V')

In [ ]:
LPV_SMC = pd.concat([SMC3_I,SMC3_V])
  
LPV_SMC['class'] = 'LPV'  
savefile(LPV_SMC,'LPV_SMC')
LPV_SMC.head()

,time,value,id,kind,class
0,2088.91535,14.977,III-OGLE-SMC-LPV-00001,I,LPV
1,2090.87535,14.978,III-OGLE-SMC-LPV-00001,I,LPV
2,2103.92958,14.980,III-OGLE-SMC-LPV-00001,I,LPV
3,2104.94812,14.986,III-OGLE-SMC-LPV-00001,I,LPV
4,2106.84596,14.998,III-OGLE-SMC-LPV-00001,I,LPV


In [ ]:
#SMC3 = pd.concat([SMC3_I,SMC3_V])
#LMC3 = pd.concat([LMC3_I,LMC3_V])
#LMC4 = pd.concat([LMC4_I,LMC4_V])
#SMC4 = pd.concat([SMC4_I,SMC4_V])

#LPV['class'] = 'LPV' 

LPV = pd.concat([LPV_LMC,LPV_SMC])

savefile(LPV,'LPV')
LPV.head()

,time,value,id,kind,class
0,2171.86025,14.956,III-OGLE-LMC-LPV-00001,I,LPV
1,2183.81956,14.961,III-OGLE-LMC-LPV-00001,I,LPV
2,2184.81907,14.961,III-OGLE-LMC-LPV-00001,I,LPV
3,2188.86966,14.970,III-OGLE-LMC-LPV-00001,I,LPV
4,2191.78070,14.955,III-OGLE-LMC-LPV-00001,I,LPV


In [ ]:
zipObj = ZipFile('LPV.zip', 'w', zipfile.ZIP_BZIP2)

zipObj.write('/content/LPV')
zipObj.close()

# Feature Extraction Function

In [ ]:
Archive('/content/drive/MyDrive/DPV.rar').extractall('/content/drive/MyDrive')

In [ ]:
def feat_ex(X,cls):
  m = extract_features(X, column_id="id", column_sort="time",
                       column_kind="kind", column_value="value")
  m['class'] = cls
  return m

In [ ]:
DPV = pd.read_csv('/content/drive/MyDrive/DPV', index_col=0, header=0 )

DPV.head()

,time,value,id,kind,class
0,2162.87556,18.178,III-OGLE-LMC-DPV-001,I,DPV
1,2171.84219,18.235,III-OGLE-LMC-DPV-001,I,DPV
2,2184.80107,18.117,III-OGLE-LMC-DPV-001,I,DPV
3,2188.85166,18.148,III-OGLE-LMC-DPV-001,I,DPV
4,2200.78375,18.223,III-OGLE-LMC-DPV-001,I,DPV


In [ ]:
DPV_f = feat_ex(DPV,'DPV')     # Extracting features
savefile(DPV_f,'DPV_f')
DPV_f.head()




Feature Extraction:   0%|          | 0/5 [00:00<?, ?it/s]


Feature Extraction:  20%|██        | 1/5 [00:32<02:11, 32.98s/it]


Feature Extraction:  40%|████      | 2/5 [01:07<01:40, 33.53s/it]


Feature Extraction:  60%|██████    | 3/5 [01:46<01:10, 35.22s/it]


Feature Extraction:  80%|████████  | 4/5 [02:09<00:31, 31.30s/it]


Feature Extraction: 100%|██████████| 5/5 [02:27<00:00, 29.56s/it]


,I__variance_larger_than_standard_deviation,I__has_duplicate_max,I__has_duplicate_min,I__has_duplicate,I__sum_values,I__abs_energy,I__mean_abs_change,I__mean_change,I__mean_second_derivative_central,I__median,I__mean,I__length,I__standard_deviation,I__variation_coefficient,I__variance,I__skewness,I__kurtosis,I__root_mean_square,I__absolute_sum_of_changes,I__longest_strike_below_mean,I__longest_strike_above_mean,I__count_above_mean,I__count_below_mean,I__last_location_of_maximum,I__first_location_of_maximum,I__last_location_of_minimum,I__first_location_of_minimum,I__percentage_of_reoccurring_values_to_all_values,I__percentage_of_reoccurring_datapoints_to_all_datapoints,I__sum_of_reoccurring_values,I__sum_of_reoccurring_data_points,I__ratio_value_number_to_time_series_length,I__sample_entropy,I__maximum,I__minimum,I__benford_correlation,I__time_reversal_asymmetry_statistic__lag_1,I__time_reversal_asymmetry_statistic__lag_2,I__time_reversal_asymmetry_statistic__lag_3,I__c3__lag_1,...,V__energy_ratio_by_chunks__num_segments_10__segment_focus_5,V__energy_ratio_by_chunks__num_segments_10__segment_focus_6,V__energy_ratio_by_chunks__num_segments_10__segment_focus_7,V__energy_ratio_by_chunks__num_segments_10__segment_focus_8,V__energy_ratio_by_chunks__num_segments_10__segment_focus_9,V__ratio_beyond_r_sigma__r_0.5,V__ratio_beyond_r_sigma__r_1,V__ratio_beyond_r_sigma__r_1.5,V__ratio_beyond_r_sigma__r_2,V__ratio_beyond_r_sigma__r_2.5,V__ratio_beyond_r_sigma__r_3,V__ratio_beyond_r_sigma__r_5,V__ratio_beyond_r_sigma__r_6,V__ratio_beyond_r_sigma__r_7,V__ratio_beyond_r_sigma__r_10,V__count_above__t_0,V__count_below__t_0,V__lempel_ziv_complexity__bins_2,V__lempel_ziv_complexity__bins_3,V__lempel_ziv_complexity__bins_5,V__lempel_ziv_complexity__bins_10,V__lempel_ziv_complexity__bins_100,V__fourier_entropy__bins_2,V__fourier_entropy__bins_3,V__fourier_entropy__bins_5,V__fourier_entropy__bins_10,V__fourier_entropy__bins_100,V__permutation_entropy__dimension_3__tau_1,V__permutation_entropy__dimension_4__tau_1,V__permutation_entropy__dimension_5__tau_1,V__permutation_entropy__dimension_6__tau_1,V__permutation_entropy__dimension_7__tau_1,V__query_similarity_count__query_None__threshold_0.0,"V__matrix_profile__feature_""min""__threshold_0.98","V__matrix_profile__feature_""max""__threshold_0.98","V__matrix_profile__feature_""mean""__threshold_0.98","V__matrix_profile__feature_""median""__threshold_0.98","V__matrix_profile__feature_""25""__threshold_0.98","V__matrix_profile__feature_""75""__threshold_0.98",class
III-OGLE-LMC-DPV-001,0.0,0.0,0.0,1.0,5716.824,102463.794094,0.055547,-0.000113,-0.000073,17.904,17.921078,319.0,0.195106,0.010887,0.038066,0.274073,-0.683622,17.922140,17.664,66.0,35.0,150.0,169.0,0.435737,0.432602,0.705329,0.702194,0.282700,0.467085,1199.267,2667.409,0.742947,1.181094,18.404,17.544,0.864123,-0.291581,-0.445167,-1.667016,5755.979815,...,0.081615,0.081423,0.081587,0.083913,0.084597,0.583333,0.416667,0.083333,0.083333,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.250000,0.375000,0.541667,0.583333,0.916667,0.271189,0.535961,0.535961,0.793730,2.204785,1.526775,2.309633,2.718473,2.770974,2.813355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DPV
III-OGLE-LMC-DPV-002,0.0,0.0,0.0,1.0,7273.977,123915.282139,0.037932,-0.000120,0.000182,17.033,17.035075,427.0,0.077108,0.004526,0.005946,0.054978,-1.057133,17.035249,16.159,25.0,23.0,210.0,217.0,0.974239,0.971897,0.526932,0.524590,0.547511,0.765808,2061.156,5570.034,0.517564,1.546819,17.197,16.881,0.864123,-0.139739,-0.244980,-0.511772,4943.883806,...,0.087593,0.087688,0.089098,0.088336,0.087322,0.735294,0.352941,0.117647,0.029412,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.235294,0.352941,0.500000,0.558824,0.823529,0.214559,0.214559,0.425848,0.633663,2.120208,1.766690,2.774161,3.216358,3.367296,3.332205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DPV
III-OGLE-LMC-DPV-003,0.0,0.0,1.0,1.0,7357.673,124736.256913,0.023968,-0.000157,-0.000021,16.953,16.953164,434.0,0.031215,0.001841,0.000974,0.129072,-0.139903,16.953192,10.378,2

# Outputs Encoding

> * **One-Hot Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

a = np.array(data['class'])
OH_Enc = OneHotEncoder( )
OH_Encoded = OH_Enc.fit_transform(a.reshape(-1,1)).toarray()
OH_Outputs = pd.DataFrame(OH_Encoded)
OH_Outputs.columns = ['OH1','OH2','OH3','OH4','OH5']

data = pd.concat([data,OH_Outputs], axis=1)
data.head()

#a = OH_Enc.inverse_transform(np.array([[1, 0, 0, 0, 0],[0, 1, 0, 0, 0],[0, 0, 1, 0, 0]
             #             ,[0, 0, 0, 1, 0],[0, 0, 0, 0, 1]]))

#print('[1, 0, 0, 0, 0] =',a[0],'\n')
#print('[0, 1, 0, 0, 0] =',a[1],'\n')
#print('[0, 0, 1, 0, 0] =',a[2],'\n')
#print('[0, 0, 0, 1, 0] =',a[3],'\n')
#print('[0, 0, 0, 0, 1] =',a[4],'\n')

> * **Ordinal Encoding**

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(data['class'])
Ordinal_Y = le.transform(data['class']) 
Y_O = pd.DataFrame(Ordinal_Y)
Y_O.columns = ['Y']

data = pd.concat([data,Y_O], axis=1)
data.head()

#le.inverse_transform([0, 1, 2, 3, 4])

# Shuffle and Imputation

> * Shuffle

In [ ]:
from sklearn.utils import shuffle

data = shuffle(data, random_state=42)
data = data.reset_index()

X = data.loc[:,'Jmag (1.22u)':'160u flux']
Y = data.loc[:,'y1':'y5']
categories = data['class']

data.head()

> * Iterative Imputer

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer()
imp_data = imp.fit_transform(X)
X = pd.DataFrame(imp_data, columns= X.columns)